# 🏆 Final Ensemble: Tree Models + LSTM

이 노트는 프로젝트의 **최종 단계**입니다.
기존의 강력한 **트리 모델(XGBoost, LightGBM, CatBoost)**과,
새롭게 구축한 **딥러닝 모델(LSTM)**의 예측값을 결합하여 최고의 성능을 도출합니다.

### 🧪 앙상블 전략
- **다양성(Diversity)**: 서로 다른 알고리즘(트리 vs 신경망)을 섞을 때 앙상블 효과가 가장 큽니다.
- **가중 평균(Weighted Blending)**: 단순 평균보다는 성능이 검증된 모델에 가중치를 더 줄 수 있습니다.
    - 일단은 **Equal Weights** (1:1:1:1)로 시작하여 안정성을 확보합니다.

In [1]:
import pandas as pd
import numpy as np
import os
import warnings

warnings.filterwarnings('ignore')

# 파일 경로 설정
OUTPUT_DIR = '../3_Output'

# 앙상블 할 파일 목록
files = {
    'xgb': 'submission_xgb_optuna.csv',
    'lgbm': 'submission_lgbm_optuna.csv',
    'cat': 'submission_cat_optuna.csv',
    'lstm': 'submission_lstm.csv'
}

preds = {}

print("Loading Submission Files...")
for model_name, filename in files.items():
    path = os.path.join(OUTPUT_DIR, filename)
    if os.path.exists(path):
        df = pd.read_csv(path)
        # 정렬 보장 (game_episode 기준)
        # sample_submission을 기준으로 merge하는 것이 안전
        preds[model_name] = df.set_index('game_episode')
        print(f" [{model_name}] Loaded: {df.shape}")
    else:
        print(f" [WARNING] File Not Found: {filename}")

# 공통 인덱스 확인 (모든 파일이 같은 shape이어야 함)
first_model = list(preds.keys())[0]
common_index = preds[first_model].index
print(f"\nTotal Predictions: {len(common_index)}")

Loading Submission Files...
 [xgb] Loaded: (2414, 3)
 [lgbm] Loaded: (2414, 3)
 [cat] Loaded: (2414, 3)
 [lstm] Loaded: (2414, 3)

Total Predictions: 2414


## Weighted Blending
각 모델의 예측값을 가중 평균합니다.

In [2]:
# 가중치 설정
# Tree 모델 3개 + LSTM 1개
# 다양성을 위해 LSTM에 충분한 비중을 줍니다.
weights = {
    'xgb': 0.25,
    'lgbm': 0.25,
    'cat': 0.25,
    'lstm': 0.25
}

# 초기화
final_x = np.zeros(len(common_index))
final_y = np.zeros(len(common_index))
total_weight = 0

for model_name, w in weights.items():
    if model_name in preds:
        print(f" Blending {model_name} (weight: {w})")
        # 인덱스 순서 맞춰서 가져오기
        df = preds[model_name].loc[common_index]
        
        final_x += df['end_x'].values * w
        final_y += df['end_y'].values * w
        total_weight += w
    else:
        print(f" Skipping {model_name} (Missing)")

# 정규화 (가중치 합으로 나누기)
if total_weight > 0:
    final_x /= total_weight
    final_y /= total_weight

# 최종 데이터프레임
final_submission = pd.DataFrame({
    'game_episode': common_index,
    'end_x': final_x,
    'end_y': final_y
}).reset_index()

# Clipping (0~105, 0~68)
final_submission['end_x'] = np.clip(final_submission['end_x'], 0, 105)
final_submission['end_y'] = np.clip(final_submission['end_y'], 0, 68)

save_path = os.path.join(OUTPUT_DIR, 'final_submission_ensemble_lstm.csv')
final_submission.to_csv(save_path, index=False)

print(f"\n✅ Final Ensemble Saved to: {save_path}")
final_submission.head()

 Blending xgb (weight: 0.25)
 Blending lgbm (weight: 0.25)
 Blending cat (weight: 0.25)
 Blending lstm (weight: 0.25)

✅ Final Ensemble Saved to: ../3_Output\final_submission_ensemble_lstm.csv


,index,game_episode,end_x,end_y
0,0,153363_1,65.847301,12.433496
1,1,153363_2,33.988996,53.702587
2,2,153363_6,35.317889,59.145740
3,3,153363_7,54.779391,9.796824
4,4,153363_8,83.250935,19.368781
